In [1]:
import pandas as pd
import rdkit.Chem
from rdkit.Chem import PandasTools as pt
from flame_help import *
import os
import glob
import copy
import numpy as np
import itertools
from flame.util import *
from flame_help import *
import flame.manage

In [2]:
#var = pd.read_csv("variables.tsv", '\t')

In [160]:
# Read datasets

rank_most = pd.read_csv('predictions_mostdilirank_0.15.tsv', sep='\t')
#rank_less = pd.read_csv('', sep='\t')
mul = pd.read_csv('predictions_mulliner_0.15.tsv', sep='\t')
pfizer = pd.read_csv('predictions_pfizer_0.15.tsv', sep='\t')
obrien = pd.read_csv('predictions_obrien_0.15.tsv', sep='\t')
# rank_most = rank_most.rename(columns={"vDILI" : "DILI"})
# #rank_less = rank_less.rename(columns={"lessdilirank" : "DILI"})
# mul = mul.rename(columns={"vDILI" : "DILI"})
# pfizer = pfizer.rename(columns={"activity" : "DILI"})
# obrien = obrien.rename(columns={"DILI positive/negative as observed" : "DILI"})

#Sort the columns propertly
columns= []
columns = list(rank_most.columns[:]) #  + list(rank_most.columns[:])

rank_most = rank_most[columns]
#rank_less = rank_less[columns]
pfizer = pfizer[columns]
mul = mul[columns]
obrien = obrien[columns]

In [98]:
db_sdf = pt.LoadSDF('Drugbank_plus_mulliner_dilirank_pfizer_obrien_nooverlap.sdf')

In [99]:
negatives = db[db_sdf['origin'] == 'drugbank']

In [100]:
db = pd.read_csv('predictions_drugbank_0.15.tsv', sep='\t')

In [147]:
pfizer = pd.concat((pfizer, negatives[37:74]), axis=0)
#rank_most = pd.concat((rank_most, negatives), axis=0)
#obrien = pd.concat((obrien, negatives), axis=0)
#mul = pd.concat((mul, negatives[306:612]), axis=0)

In [76]:
mul.shape

(2965, 13)

In [183]:
par = {'model' : ['RF'],
           'conformalSignificance': [0.10, 0.15, 0.20],
           'imbalance' : ["simple_subsampling"],
           'quantitative' : [False],
           'tune' : [True],
           'conformal' : [True],
           'computeMD_method' : [['RDKit_md']],
           'input_type' : ['data'], 
           'TSV_activity' : ['DILI'],
           'TSV_objnames' : [True],
           'TSV_varname' : [True],
           'modelAutoscaling' : ['StandardScaler'],
            'feature_selection': ['Kbest'],
            'feature_number' : [3, 5, 7, 9 ,11]}
it = itertools.product(par['model'], par['conformalSignificance'], par['imbalance'],
                      par['quantitative'], par['tune'], par['conformal'], 
                      par['computeMD_method'], par['input_type'], par['TSV_activity'], par['TSV_objnames'],
                      par['TSV_varname'], par['modelAutoscaling'], par['feature_selection'],
                      par['feature_number'])

In [184]:
config = ""
for index, config in enumerate(copy.copy(it)):
    config = dict(zip(par.keys(), list(config)))


In [185]:
databases =  {'rank_most' : rank_most,
                # 'rank_less' : rank_less,
                  'pfizer' : pfizer,
                 'obrien' : obrien,
                 'mulliner' : mul}

In [186]:
def remove_duplicates(database2, database):
    database2 = database2[~database2['parent_nonstd_inkey'].isin(database['parent_nonstd_inkey'])]
    return database2

In [202]:
columns_val = ['combina', 'endpoint', 'version', 'model', 'TP', 'TN', 'FP', 'FN',
           'Sensitivity', 'specificity', 'MCC', 'Coverage', 'accuracy']

for name, database in databases.items():
    print(database.shape)
    if name == 'pfizer':
        ext_val = []
        for index, config in enumerate(list(copy.copy(it))[4:5]):
            print(config)
            config = dict(zip(par.keys(), list(config)))
            # Write the tsv
            name_file = name + '_meta_val.tsv'
            database.to_csv("./metamodels/" + name_file, sep='\t', index=False)
            # build model
            action_new(name)
            build_model(name, config, 'metamodels/' + name_file)
            print(name)
            # Run external predicitions
            for name2, database2 in databases.items():
                if name2 != name:
                    print(name2)

                    database2 = remove_duplicates(database2, database)
                    # save external prediction set
                    name_file2 = name + "_test_on_" + name2 + ".tsv"
                    database2.to_csv("./metamodels/test_sets/" + name_file2, sep='\t', index=False)
                    pred = Predict(name, 0)
                    results = pred.run("./metamodels/test_sets/" + name_file2)
                    pred_results = get_quality(results)
                    ext_val.append([index, name2, 0, "RF"] + pred_results)

        validation = pd.DataFrame(ext_val, columns=columns_val).sort_values(by=['endpoint', 'MCC'], ascending=[False, False])       
        validation.to_csv("./metamodels/" +name + "_ev.tsv", sep='\t', index=False)
        
        

            
    
    

INFO - Running with input type: data
INFO - Building model using internal toolkit : Sci-kit learn
INFO - Subsampling of positive instances
INFO - simple_subsampling performed
INFO - Number of objects after sampling: 110
INFO - Data scaled using StandarScaler
INFO - Variable selection applied, number of final variables:11
INFO - cv is: LeaveOneOut()
INFO - Starting model building
INFO - Computing best hyperparameter values


(177, 13)
(147, 13)
('RF', 0.1, 'simple_subsampling', False, True, True, ['RDKit_md'], 'data', 'DILI', True, True, 'StandardScaler', 'Kbest', 11)


INFO - best parameters: , {'class_weight': None, 'max_features': 'sqrt', 'n_estimators': 50}
INFO - Building aggregated conformal RF model
INFO - Starting model validation
INFO - Model finished successfully
INFO - Running with input type: data


       nobj ( number of objects ) : 110.0
       nvarx ( number of predictor variables ) : 11.0
       model ( model type ) : RF qualitative (optimized)
       model ( model type ) : conformal RF qualitative
       TP ( True positives in cross-validation ) : 0.0
       TN ( True negatives in cross-validation ) : 3.0
       FP ( False positives in cross-validation ) : 0.0
       FN ( False negatives in cross-validation ) : 5.0
       Sensitivity ( Sensitivity in cross-validation ) : 0.0
       Specificity ( Specificity in cross-validation ) : 1.0
       MCC ( Matthews Correlation Coefficient in cross-validation ) : nan
       Conformal_coverage ( Conformal coverage ) : 0.0727
       Conformal_accuracy ( Conformal accuracy ) : 0.375
pfizer
rank_most


INFO - Running with input type: data


       obj_num ( number of objects ) : 153.0
       TP ( True positives in external-validation ) : 6.0
       TN ( True negatives in external-validation ) : 12.0
       FP ( False positives in external-validation ) : 2.0
       FN ( False negatives in external-validation ) : 4.0
       Sensitivity ( Sensitivity in external-validation ) : 0.6
       Specificity ( Specificity in external-validation ) : 0.8571
       MCC ( Mattews Correlation Coefficient in external-validation ) : 0.4781
       Conformal_coverage ( Conformal coverage in external-validation ) : 0.1558
       Conformal_accuracy ( Conformal accuracy in external-validation ) : 0.75
obrien
       obj_num ( number of objects ) : 22.0
       TP ( True positives in external-validation ) : 2.0
       TN ( True negatives in external-validation ) : 1.0
       FP ( False positives in external-validation ) : 0.0
       FN ( False negatives in external-validation ) : 0.0
       Sensitivity ( Sensitivity in external-validation ) : 1.0
 

INFO - Running with input type: data


       obj_num ( number of objects ) : 1500.0
       TP ( True positives in external-validation ) : 104.0
       TN ( True negatives in external-validation ) : 47.0
       FP ( False positives in external-validation ) : 42.0
       FN ( False negatives in external-validation ) : 41.0
       Sensitivity ( Sensitivity in external-validation ) : 0.7172
       Specificity ( Specificity in external-validation ) : 0.5281
       MCC ( Mattews Correlation Coefficient in external-validation ) : 0.2459
       Conformal_coverage ( Conformal coverage in external-validation ) : 0.1559
       Conformal_accuracy ( Conformal accuracy in external-validation ) : 0.6453
(40, 13)
(1560, 13)


In [199]:
dbank.head()

,parent_nonstd_inkey,Jaundice_cholestatic,Hyperbilirubinaemia,Hepatic_failure,Cholelithiasis,Cholestasis,Hepatitis,Jaundice,Hepatobiliary_disease,Hepatomegaly,Hepatotoxicity,Hepatic_function_abnormal,DILI
0,AAIMHYPMAWJDBG-UHFFFAOYNA-N,0,2,2,2,2,2,2,2,2,0,2,0.0
1,AAKJLRGGTJKAMG-IJFJLPMKNA-O,2,2,2,2,2,2,0,1,2,2,0,1.0
2,AAKJLRGGTJKAMG-LNNLXFCONA-N,2,2,2,2,2,1,0,1,2,1,0,1.0
3,AAOVKJBEBIDNHE-UHFFFAOYNA-N,0,0,0,2,0,2,1,1,2,0,1,1.0
4,AAQOQKQBGPPFNS-UHFFFAOYNA-N,0,2,0,2,2,2,2,0,0,0,2,0.0


In [205]:
dbank = pd.read_csv('predictions_drugbank_0.15.tsv', '\t')

drugbank = []
for name, database in databases.items():
    print(name)
    if name != 'obrien':

        database2 = remove_duplicates(dbank, database)
        # save external prediction set
        name_file2 = name + "_test_on_drugbank.tsv"
        database2.to_csv("./metamodels/test_sets/" + name_file2, sep='\t', index=False)
        pred = Predict(name, 0)
        results = pred.run("./metamodels/test_sets/" + name_file2)
        pred_results = get_quality(results)
        drugbank.append([name, 0, 'RF'])


rank_most


INFO - Running with input type: data
INFO - Running with input type: data


       obj_num ( number of objects ) : 3917.0
       TP ( True positives in external-validation ) : 228.0
       TN ( True negatives in external-validation ) : 537.0
       FP ( False positives in external-validation ) : 174.0
       FN ( False negatives in external-validation ) : 187.0
       Sensitivity ( Sensitivity in external-validation ) : 0.5494
       Specificity ( Specificity in external-validation ) : 0.7553
       MCC ( Mattews Correlation Coefficient in external-validation ) : 0.3068
       Conformal_coverage ( Conformal coverage in external-validation ) : 0.2874
       Conformal_accuracy ( Conformal accuracy in external-validation ) : 0.6794
pfizer


INFO - Running with input type: data


       obj_num ( number of objects ) : 3918.0
       TP ( True positives in external-validation ) : 130.0
       TN ( True negatives in external-validation ) : 138.0
       FP ( False positives in external-validation ) : 101.0
       FN ( False negatives in external-validation ) : 68.0
       Sensitivity ( Sensitivity in external-validation ) : 0.6566
       Specificity ( Specificity in external-validation ) : 0.5774
       MCC ( Mattews Correlation Coefficient in external-validation ) : 0.2333
       Conformal_coverage ( Conformal coverage in external-validation ) : 0.1115
       Conformal_accuracy ( Conformal accuracy in external-validation ) : 0.6133
obrien
mulliner
       obj_num ( number of objects ) : 2505.0
       TP ( True positives in external-validation ) : 271.0
       TN ( True negatives in external-validation ) : 521.0
       FP ( False positives in external-validation ) : 269.0
       FN ( False negatives in external-validation ) : 123.0
       Sensitivity ( Sensitivity i

In [208]:
dbank['DILI'].value_counts()


0.0    2301
1.0    1765
Name: DILI, dtype: int64

In [ ]:
#pred = Predict('ltkb', 0)
#results = pred.run("./metamodels/test_sets/pfizer_test_on_rank_most.tsv")

In [206]:
models = []
for key in databases.keys():
    models.append([key, 0])
cross_val = get_cross_val_stats([['rank_most',0], ['pfizer', 0], ['mulliner', 0]])
    

In [207]:
cross_val.to_csv('./metamodels/cross_val_stats.tsv', sep='\t')

In [ ]:
results = pd.read_csv('metamodels/etransafe/output.tsv', sep='\t',index_col=None)

In [ ]:
final = []
for index, row in results.iterrows():
    c0 = row['c0']
    c1 = row['c1']
    name = row['obj_nam']
    val = "negative"
    ad = "YES"
    if c0 == c1:
        val = "out of domain"
        ad = "NO"
    else:
        if c0:
            val = val
            
        else:
            val = 'positive'
    final.append([name, val, "pseudo confidence interval 85%", ad])
frame = pd.DataFrame(final, columns=['ID', 'Prediction', 'Probability(POS)', 'In Applicability Domain?', 'In Training Set?'])


In [ ]:
frame['Prediction'].value_counts()

In [ ]:
frame.to_csv('metamodels/etransafe/ai_test_results.tsv', sep='\t', index=False)

In [ ]:
exam = pd.read_csv('metamodels/etransafe/ai_example.csv', sep='\t')
exam.shape

In [ ]:
subframe = exam[['ID', 'SMILES', "In Training Set?"]].reset_index(drop=True)

In [ ]:
frame = frame.reset_index(drop=True)

In [ ]:
frame2 = pd.merge(frame, subframe, on='ID', how='right')

In [ ]:
frame2.to_csv("metamodels/etransafe/ai_meta_predictions.tsv", sep='\t', index=False)

In [279]:
# Ambiguous DILI concern

results = pd.read_csv('metamodels/ambiguous/output.tsv', sep='\t',index_col=None)

In [280]:
results.tail()

,obj_nam,c0,c1,ymatrix,Unnamed: 4
50,trimethobenzamide,True,False,0.0,NaN
51,vardenafil,False,True,0.0,NaN
52,verteporfin,True,True,0.0,NaN
53,vinorelbine,False,True,0.0,NaN
54,zolmitriptan,True,True,0.0,NaN


In [281]:
final = []
for index, row in results.iterrows():
    c0 = row['c0']
    c1 = row['c1']
    name = row['obj_nam']
    val = "negative"
    ad = "YES"
    if c0 == c1:
        val = "out of domain"
        ad = "NO"
    else:
        if c0:
            val = val
            
        else:
            val = 'positive'
    final.append([name, val])
frame = pd.DataFrame(final, columns=['ID', 'Prediction'])


In [282]:
mulli = frame

In [283]:
mulli['Prediction'].value_counts()

out of domain    24
negative         16
positive         15
Name: Prediction, dtype: int64

In [284]:
frame_p = pd.DataFrame()
frame_p['name'] = rankP['ID']
frame_p['DILIrank_most'] = rankP['Prediction']
frame_p['Mulliner'] = mulli['Prediction']

In [285]:
frame_p

,name,DILIrank_most,Mulliner
0,amantadine,out of domain,negative
1,amoxapine,out of domain,negative
2,anagrelide,out of domain,out of domain
3,aprepitant,out of domain,out of domain
4,betaxolol,negative,negative
5,bisoprolol,negative,negative
6,bumetanide,out of domain,out of domain
7,chlorthalidone,out of domain,positive
8,cilostazol,positive,out of domain
9,clonidine,positive,out of domain


In [286]:
frame_p.to_csv('ambiguous_predictions.csv', sep='\t', index_label=None)